In [1]:
import unicodedata
import re
import pandas as pd

# =====================================================================

# ファイルを開いて全文、タイトル、著者を読み込む
with open('data/hashire_merosu.txt', mode='r', encoding='UTF-8') as f:
    title = f.readline()
    author = f.readline()
    content = f.read()

# 改行を無くし全文をつなげる
content = ' '.join(content.split())
# 文章を正規化する
content = unicodedata.normalize('NFKC', content)

# 本文以外に該当する部分を切り離す
pattern = re.compile(r'^.+(#地から1字上げ].+#地から1字上げ]).+$')
body =re.match(pattern, content).group(1)

# 本文のみにクリーニングする
body = body.replace('#地から1字上げ] -------------------------------------------------------', '')
body = body.replace(' [#地から1字上げ]', '')

# タイトルと著者を別途取って来る(改行を除外)
title = title.replace('\n', '')

author = author.replace('\n', '')

# =====================================================================

# ファイルを開いて全文を読み込む
with open('data/hashire_merosu.txt', mode='r', encoding='UTF-8') as f:
    content = f.readlines()

# データフレームに全文を1行ずつ入れる
df = pd.DataFrame(content, columns=['text'])
df['text'] = df['text'].str.replace('\n', '')

# 公開日と修正日について
date = df[(df['text'].str.contains('日公開'))|(df['text'].str.contains('日修正'))].copy()

date['text'] = date['text'].str.replace('公開', '')
date['text'] = date['text'].str.replace('修正', '')

date['text'] = date['text'].str.replace('年', '/')
date['text'] = date['text'].str.replace('月', '/')
date['text'] = date['text'].str.replace('日', '')

date['text'] = pd.to_datetime(date['text'])
date.dtypes

# 取得した日時と計算結果
release_date = date.iat[0, 0]
update_date = date.iat[1, 0]
# print(release_date)
# print(update_date)

date = update_date - release_date


# 取得項目を格納したデータフレーム
booklist = pd.DataFrame([[title, author, release_date, update_date, body]],
columns = ['title', 'author', 'release_date', 'update_date', 'body'])

print(booklist)

   title author release_date update_date  \
0  走れメロス    太宰治   2000-12-04  2011-01-17   

                                                body  
0   メロスは激怒した。必ず、かの邪智暴虐《じゃちぼうぎゃく》の王を除かなければならぬと決意した...  


C:\Users\ryoun\AppData\Local\Temp\ipykernel_18856\986321221.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
booklist

,title,author,release_date,update_date,body
0,走れメロス,太宰治,2000-12-04,2011-01-17,メロスは激怒した。必ず、かの邪智暴虐《じゃちぼうぎゃく》の王を除かなければならぬと決意した...


In [5]:
import MeCab
tagger = MeCab.Tagger()
body = booklist.iloc[0, 4]
parsed = tagger.parse(body).split('\n')
parsed[:4]

['メロス\t名詞,一般,*,*,*,*,*',
 'は\t助詞,係助詞,*,*,*,*,は,ハ,ワ',
 '激怒\t名詞,サ変接続,*,*,*,*,激怒,ゲキド,ゲキド',
 'し\t動詞,自立,*,*,サ変・スル,連用形,する,シ,シ']

In [7]:
parsed[-4:]

['た\t助動詞,*,*,*,特殊・タ,基本形,た,タ,タ', '。\t記号,句点,*,*,*,*,。,。,。', 'EOS', '']

In [8]:
parsed = parsed[:-2]
parsed[-4:]

['赤面\t名詞,サ変接続,*,*,*,*,赤面,セキメン,セキメン',
 'し\t動詞,自立,*,*,サ変・スル,連用形,する,シ,シ',
 'た\t助動詞,*,*,*,特殊・タ,基本形,た,タ,タ',
 '。\t記号,句点,*,*,*,*,。,。,。']

In [11]:
import re

In [12]:
*values, = map(lambda s: re.split(r'\t|,', s), parsed)
values[:4]

AttributeError: module 're' has no attribute 'splite'